# Using the CVXOPT Package to solve SVM Problems

CVXOPT is a free solver system for constrained optimistion problems (http://cvxopt.org) with an interface written in Python.

The interface is similar to that provided by Numpy, the only issue being that CVXOPT has its own notion of matrices and vectors, so programs using CVXOPT have to employ these CVXOPT versions.  Luckily there is a good interface to Numpy, so Numpy array objects, along with standard Python lists, can easily be translated to CVXOPT matrices.

The most important CVXOPT imports are "matrix", for matrix construction, and "solvers".

The CVXOPT "mul" function is useful if you want ot do element by element multiplication of matrices (we'll use this below).

In [ ]:
from cvxopt import matrix, solvers, mul

Use matrix to build a matrix.  The only issue is that the arguments are in column-major order (the opposite to the Numpy convention).  To create a doube-precision float matrix, make sure that at least one of the entries has a decimal pint (or include the keyword argument "tc='d'").

In [ ]:
A = matrix([[2.,0,3],[0,1,4]])   # Creates a 3 row, 2 column matrix.

In [ ]:
A                                # Just gives shape and type, not contents.

In [ ]:
print(A)                         # Explicit print needed to view contents. Note column-major.

In [ ]:
print(matrix(5.0))               # A 1x1 matrix can be created using a single scalar

In [ ]:
vec = matrix([1,1], tc='d')      # Column vectors by default (Nx1 matrices).

In [ ]:
vec

In [ ]:
print(vec)

In [ ]:
print(A*vec)                     # '*' is matrix multiplication in CVXOPT, no "dot".

In [ ]:
print(vec.T)                    # To make a row-vector, transpose a column-vector.

In [ ]:
vec.T                           # Shape of transpose: 1 row, 2 columns.

In [ ]:
print(matrix([[1],[2]]))        # Or you can do this!  Not recommended.

### Using the 'qp' solver.

Here's a manual run-through of building a linear SVM to solve the 2-input OR problem.

First create a vector of the desired outputs.  Note that the vector is forced to be a
float one because of the decimal point in the first entry.  Alternatively, specify "tc='d'" as
a keyword in the constructor.

It is important to use double-precision float matrices everywhere here, as the "qp" solver
expects all its arguments to be matrices of double-precision numbers.

In [ ]:
t=matrix([-1.,1,1,1])

It can be useful to create a cross-product matrix $t_it_j$ of the training inputs and multiply it elementwise by a cross-product matrix of input vectors to create the CVXOPT $P$ matrix.

In [ ]:
print(t*t.T)

Input vectors $\vec{x_i}$, One vector per row of the $X$ matrix.  Note the use of "tc='d'" keyword to the constructor in order to force double-precision.

In [ ]:
X=matrix([[0,0,1,1],[0,1,0,1]], tc='d')

In [ ]:
print(X)

Cross-product matrix of input vectors $\vec{x_i}.\vec{x_j}$.  

For a kernel
SVM, you would need to build an $N \times N$ matrix of the results of applying the kernel
function, $K(\vec{x_i}.\vec{x_j})$.

In [ ]:
print(X*X.T)

The $P$ matrix to input to the qp solver is the elementwise product of these two cross-product matrices.  In CVXOPT, the routine "mul" provides element by element multiplication of two matrices.

In [ ]:
P=mul(t*t.T,X*X.T)

In [ ]:
print(P)

Now construct all the other required matrices for the solver.

The $\vec{q }$ vector is supplied to the solver as a <i>column</i> vector of $-1$'s with $N$ elements  (even though the equation for $Q$ shows a row vector, the solver expects this to be supplied as a column vector!).

In [ ]:
q=matrix(4*[-1.0])      #  4*[-1.0] makes a 4-element Python list containing -1's.

For the hard-margin SVM needed for 2-input OR, the $G$ matrix, needed to handle the constraints $\lambda_i \geq 0$, is a $4 \times 4$ matrix with $-1$'s on its main diagonal and $0$'s elsewhere.  The Numpy "eye" function can help with this.

Note that the constraints $\lambda_i \geq 0$ have to be recast to $-\lambda_i \leq 0$ because the qp solver expects its inequality contraints in a matrix-vector equation $G \vec\lambda \leq 0$.

In [ ]:
import numpy as np

In [ ]:
G = matrix(-np.eye(4))  # Note the minus sign!  Important!

In [ ]:
G

In [ ]:
print(G)

The $\vec{h}$ vector is also needed.  For this hard-margin SVM, its just a column vector of zeros.

In [ ]:
h = matrix(4*[0.0])

Finally we can call the solver to find the Lagrange multipliers for the problem.

Note that the solver requires a matrix $A$ to handle any <i>equality</i> constraints, which
is employs like so: $A\vec{\lambda} = \vec{b}$.  

In this problem the equality constraint is $\sum t_i \lambda_i = 0$, so we can use the <i>transpose</i> of the $\vec{t}$ vector for $A$, and a 1-e,emt matrix containing $0.0$ for 
$\vec{b}$.

In [ ]:
r = solvers.qp(P,q,G,h,t.T,matrix(0.0))

If the information printed out by the solver ends with 'Optimal solution found', it has worked and found the Lagrange multipliers for the problem.

The solver returns a lot of data about its solition in a Python dictionary object, here $r$.

In [ ]:
r

The Lagrange multiplers are returned as a CVXOPT matrix under the 'x' key in the dictionary.

Note that the slack entry is not exactly zero, instead it is a very small value, close to zero.  This is an artifact of the solver's internal algorithm (a so-called "interior-point method"), and is normal behaviour.


In [ ]:
print(r['x'])

The solution status is returned as a string under the 'status' key.  String 'optimal' means a valid solution has been found.

In [ ]:
print(r['status'])

It can be convenient to convert the CVXOPT matrix containing the Lagrange multipliers to a more common Python data structure, such as a list.

In [ ]:
list(r['x'])

This representation prints out more significant digits of the solution and makes it clear that none of the Lagrange values is exact (i.e., [4,2,2,0]).  They all contain numerical noise, and this is to be expected.